This notebook shows the comparison between the Cherenkov light signal generated by CORSIKA IACT and that of CHASM for the same shower. First we import a CORSIKA IACT file using the eventio python module.

In [1]:
import numpy as np
from simulation import *
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import eventio

%matplotlib widget

#Wrap eventio for extraction of shower parameters
class EventioWrapper(eventio.IACTFile):
    def __init__(self, corsika_filename):
        super().__init__(corsika_filename)
        self.event = self.get_event()
        self.theta = self.event.header[10]
        self.phi = self.event.header[11] + np.pi #CHASM coordinate system rhat points up the axis, not down like CORSIKA
        self.obs = self.event.header[5]/100. #convert obs level to meters
        nl = self.event.longitudinal['nthick'] #number of depth steps
        self.X = np.arange(nl, dtype=float) * self.event.longitudinal['thickstep'] #create depth steps
        self.nch = np.array(self.event.longitudinal['data'][6]) #corresponding number of charged particles
        counter_x = self.telescope_positions['x']/100. # cm -> m
        counter_y = self.telescope_positions['y']/100. # cm -> m
        counter_z = self.telescope_positions['z']/100. # cm -> m
        self.counter_r = np.sqrt(counter_x**2 + counter_y**2 + counter_z**2)
        self.counter_radius = self.telescope_positions['r']/100. # cm -> m
        self.counter_vectors = np.vstack((counter_x,counter_y,counter_z)).T
        self.iact_nc = len(self.counter_r)
        self.min_l = self.event.header[57] #wavelength in nm
        self.max_l = self.event.header[58] #wavelength in nm
        self.ng_sum = np.array([self.event.n_photons[i] for i in range(len(counter_x))])
        self.percs_and_bins(self.event)
         
    def get_event(self, event_index: int = 0):
        '''This method returns an individual shower event from the CORSIKA IACT file.'''
        event_list = []
        for event in self:
            event_list.append(event)
        if len(event_list) == 1:
            return event_list[0]
        else:
            return event_list[event_index]
        
    def percs_and_bins(self, event):
        iact_gmnt = np.array([event.photon_bunches[i]['time'].min() for i in range(self.iact_nc)])
        iact_gmxt = np.array([event.photon_bunches[i]['time'].max() for i in range(self.iact_nc)])
        iact_g05t = np.array([np.percentile(event.photon_bunches[i]['time'], 5.) for i in range(self.iact_nc)])
        iact_g95t = np.array([np.percentile(event.photon_bunches[i]['time'],95.) for i in range(self.iact_nc)])
        iact_gd90 = iact_g95t-iact_g05t
        iact_g01t = np.array([np.percentile(event.photon_bunches[i]['time'], 1.) for i in range(self.iact_nc)])
        iact_g99t = np.array([np.percentile(event.photon_bunches[i]['time'],99.) for i in range(self.iact_nc)])
        self.wl = np.abs(np.array([event.photon_bunches[i]['wavelength'] for i in range(self.iact_nc)]))
        iact_ghdt = np.ones_like(iact_gmnt)
        iact_ghdt[iact_gd90<30.]   = 0.2
        iact_ghdt[iact_gd90>100.] = 5.
        self.iact_ghmn = np.floor(iact_gmnt)
        self.iact_ghmn[iact_ghdt==5.] = 5*np.floor(self.iact_ghmn[iact_ghdt==5.]/5.)
        self.iact_ghmx = np.ceil(iact_gmxt)
        self.iact_ghmx[iact_ghdt==5.] = 5*np.ceil(self.iact_ghmx[iact_ghdt==5.]/5.)
        self.iact_ghnb = ((self.iact_ghmx-self.iact_ghmn)/iact_ghdt).astype(int)
        
    def get_photon_times(self, counter_index: int, event_index: int = 0):
        '''This method returns the array of arrival times for each photon bunch for a particular
        event and counter.'''
        return self.get_event(event_index).photon_bunches[counter_index]['time']
    
    def get_photons(self, counter_index: int, event_index: int = 0):
        '''This method returns the array of the number of photons in each photon bunch for a particular
        event and counter.'''
        return self.get_event(event_index).photon_bunches[counter_index]['photons']
    
    def shower_coordinates(self):
        """
        This function returns the emmission coordinates of CORSIKA photon bunches
        and the number of photons at each emission coordinate (for each IACT)
        """
        x_e = []
        y_e = []
        z_e = []
        p_e = []
        obslevel = self.header[5][0]
        for i in range(len(self.telescope_positions)):
            pos = self.telescope_positions[i]
            x_t = np.array(self.event.photon_bunches[i]['x'])
            y_t = np.array(self.event.photon_bunches[i]['y'])
            z = np.array(self.event.photon_bunches[i]['zem'])
            cx = np.array(self.event.photon_bunches[i]['cx'])
            cy = np.array(self.event.photon_bunches[i]['cy'])
            p = np.array(self.event.photon_bunches[i]['photons'])
            z -= obslevel
            cz = np.sqrt(1 - (cx**2 + cy**2))
            x = x_t - cx * z / cz + pos[0]
            y = y_t - cy * z / cz + pos[1]
            x_e = np.append(x_e,x / 100)
            y_e = np.append(y_e,y / 100)
            z_e = np.append(z_e,z / 100)
            p_e = np.append(p_e,p)
        return np.array(x_e),np.array(y_e),np.array(z_e),np.array(p_e)
    
  
corsika_file = '/home/isaac/corsika_data/unattenuated_30degree_sealevel/iact_s_000007.dat'
att_corsika_file = '/home/isaac/corsika_data/attenuated_30degree_sealevel/iact_s_000007.dat'

cors_no_att = EventioWrapper(corsika_file) #CORSIKA shower with no atmospheric absorbtion
cors_att = EventioWrapper(att_corsika_file) #CORSIKA shower with atmospheric absorbtion

Now we create a CHASM simulation, and add elements to it using the same parameters as the CORSIKA shower.

In [2]:
sim = ShowerSimulation()

#Add shower axis
sim.add(DownwardAxis(cors_no_att.theta, cors_no_att.phi, cors_no_att.obs))

#Add shower profile
sim.add(UserShower(cors_no_att.X, cors_no_att.nch)) #profiles could also be created using Gaisser-Hillas parameters

#Add CORSIKA IACT style spherical telescopes
sim.add(SphericalCounters(cors_no_att.counter_vectors, cors_no_att.counter_radius))

#Add wavelength interval for Cherenkov yield calculation
sim.add(Yield(cors_no_att.min_l, cors_no_att.max_l))


Now the simulation has all the necessary elements to calculate a Cherenkov signal.

In [3]:
#Create objects which calculate Cherekov signal
sim.run()

#Get signal photons
ng_sum = sim.get_signal_sum()

#Get attenuated signal photons
ng_sum_att = sim.get_attenuated_signal_sum()

Now let's plot the Cherenkov signal from both CORSIKA IACT and CHASM.

In [4]:
fig = plt.figure()
plt.scatter(cors_no_att.counter_r, cors_no_att.ng_sum, c = 'k', label = 'CORSIKA IACT');
plt.scatter(cors_no_att.counter_r, cors_att.ng_sum, c = 'g', label = 'CORSIKA IACT (Attenuated)');
plt.scatter(cors_no_att.counter_r, ng_sum, c = 'r', label = 'CHASM');
plt.scatter(cors_no_att.counter_r, ng_sum_att, c = 'b', label = 'CHASM (Attenuated)');
plt.semilogy();
plt.legend();
plt.grid()
plt.title('Cherenkov Lateral Distribution');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

We can also compare the arrival time distributions of CHASM photons from each stage to that of the CORSIKA IACT photon bunches. For better modeling of the leading edge of the pulse, the CHASM mesh option samples the Cherenkov distribution at rings surrounding the axis, rather than just along the axis. Charged particles are distributed to each ring according to the universal energy-independent NKG distribution.

In [5]:
times, photons = sim.get_signal_times()
sim.run(mesh = True)
mesh_times, mesh_photons = sim.get_signal_times()

Now let's look at the arrival times for various counters at various distances from the shower core.

In [6]:
counters_2_plot = np.array([1,5,10,20,30,40,45]) #indices of counters for which to plot arrival time distributions
for counter in counters_2_plot:
    fig = plt.figure()
    plt.hist(cors_no_att.get_photon_times(counter), cors_no_att.iact_ghnb[counter], (cors_no_att.iact_ghmn[counter],cors_no_att.iact_ghmx[counter]),
                          color='k',
                          weights=cors_no_att.get_photons(counter),
                          histtype='step',label='CORSIKA-IACT no att')
    plt.hist(mesh_times[counter], cors_no_att.iact_ghnb[counter], (cors_no_att.iact_ghmn[counter],cors_no_att.iact_ghmx[counter]),
                      color='b',
                      weights=mesh_photons[counter],
                      histtype='step',label='CHASM no att mesh')
    plt.hist(times[counter], cors_no_att.iact_ghnb[counter], (cors_no_att.iact_ghmn[counter],cors_no_att.iact_ghmx[counter]),
                      color='r',
                      weights=photons[counter],
                      histtype='step',label='CHASM no att')
    plt.xlabel('Time (ns)')
    plt.ylabel('Number of Cherenkov photons')
    plt.title(f'Counter {cors_no_att.counter_r[counter]:.1f} m from Core.')
    plt.semilogx()
    plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

The leading edge of the Cherenkov pulse comes earlier when Cherenkov light is sampled from physical charged particle positions because those points are closer to the counters than the axis itself. Here we plot the CORSIKA IACT photon bunch source locations along with the spread axis source locations, each weighted by the number of photons produced there.

In [8]:
fig = plt.figure()
ax = Axes3D(fig)
for s in sim.signals[:,0]:
    ng = s.calculate_ng()[0].sum(axis=0)
    ax.scatter(s.axis.vectors[:2000,0],s.axis.vectors[:2000,1],s.axis.vectors[:2000,2],c='b', s=ng[:2000], alpha=0.2)
xb, yb, zb, pb = cors_no_att.shower_coordinates()
ax.scatter(xb,yb,zb,c='k', s=pb, alpha=0.2);
ax.scatter(cors_no_att.counter_vectors[:,0], cors_no_att.counter_vectors[:,1], cors_no_att.counter_vectors[:,2], c = 'c', s = 10);
ax.set_xlim(-8000,300);
ax.set_ylim(-300,300);
ax.set_zlim(0,5000);
ax.set_xlabel('x (m)');
ax.set_ylabel('y (m)');
ax.set_zlabel('z (m)');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …